# Import

In [68]:
import os, time, numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm

# Configuration

In [69]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_ROOT = "data/CUB_200_2011"
NUM_CLASSES = 200
INIT_LR = 1e-4
NEW_LR = 1e-6
WEIGHT_DECAY = 1e-4
BATCH_SIZE = 32
STEP_SIZE = 7
GAMMA = 0.1
EPOCHS_STAGE1 = 7
EPOCHS_STAGE2 = 150

# Dataset Setup

In [71]:

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

DATA_ROOT = "data/CUB_200_2011"
BATCH_SIZE = 32

class CUBDataset(Dataset):
    def __init__(self, root, train=True, transform=None):
        self.root = root
        self.transform = transform

        img_txt = os.path.join(root, "images.txt")
        label_txt = os.path.join(root, "image_class_labels.txt")
        split_txt = os.path.join(root, "train_test_split.txt")

        with open(img_txt) as f:
            imgs = [x.strip().split(" ") for x in f.readlines()]
        with open(label_txt) as f:
            labels = [int(x.strip().split(" ")[1]) - 1 for x in f.readlines()]
        with open(split_txt) as f:
            split = [int(x.strip().split(" ")[1]) for x in f.readlines()]

        self.samples = []
        for (img_id, img_path), label, is_train in zip(imgs, labels, split):
            if (train and is_train == 1) or (not train and is_train == 0):
                self.samples.append((os.path.join(root, "images", img_path), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label  # tuple (Tensor, int)


train_set = CUBDataset(DATA_ROOT, train=True, transform=train_transforms)
test_set = CUBDataset(DATA_ROOT, train=False, transform=test_transforms)

train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=10,          
    pin_memory=True,          
    persistent_workers=True,  
    prefetch_factor=4,        
)

test_loader = DataLoader(
    test_set,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=10,           
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4,
)

print(f"Loaded {len(train_set)} training and {len(test_set)} testing images.")


Loaded 5994 training and 5794 testing images.


# Model

In [72]:
class BilinearCNN(nn.Module):
    def __init__(self, num_classes=200, dropout_p=0.25):
        super().__init__()
        backbone = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1)
        self.features = nn.Sequential(*list(backbone.features.children())[:-1])
        self.num_features = 512
        self.fc = nn.Linear(self.num_features * self.num_features, num_classes)
        nn.init.kaiming_uniform_(self.fc.weight)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        x = self.features(x)
        B, C, H, W = x.size()
        x = x.view(B, C, H * W)
        bilinear = torch.bmm(x, x.transpose(1, 2)) / (H * W)
        bilinear = bilinear.view(B, C * C)
        bilinear = torch.sign(bilinear) * torch.sqrt(torch.abs(bilinear) + 1e-5)
        bilinear = nn.functional.normalize(bilinear)
        x = self.dropout(bilinear)
        return self.fc(x)

# Utils

In [73]:
# MixUp Augmentation
def mixup_data(x, y, alpha=0.2, device='cuda'):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# Evaluation
@torch.no_grad()
def evaluate_model(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    for images, labels in tqdm(loader, desc="Evaluating", leave=False):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Accumulate *sum* of per-sample losses
        batch_size = labels.size(0)
        total_loss += loss.item() * batch_size
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / total  
    accuracy = 100.0 * correct / total
    return accuracy, avg_loss

# Train
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs, device):
    model.train()
    for epoch in range(num_epochs):
        running_loss, correct_train,total_train= 0.0, 0.0, 0.0
        start = time.time()
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            mixed_imgs, y_a, y_b, lam = mixup_data(images, labels, alpha=0.2, device=device)
            optimizer.zero_grad()
            outputs = model(mixed_imgs)
            loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            with torch.no_grad():
                raw_outputs = model(images)  # normal forward without MixUp
                preds = raw_outputs.argmax(1)
                correct_train += (preds == labels).sum().item()
                total_train += labels.size(0)
        scheduler.step()
        epoch_loss = running_loss / len(train_loader)
        train_acc = 100 * correct_train / total_train
        acc, test_loss = evaluate_model(model, test_loader, criterion, device)
        train_losses.append(epoch_loss)
        test_losses.append(test_loss)
        test_accuracies.append(acc)
        print(f"[Epoch {epoch+1:02d}] Train Loss={epoch_loss:.4f} | Test Loss={test_loss:.4f} "
              f"| Test Acc={acc:.2f}% | LR={optimizer.param_groups[0]['lr']:.2e} | "
              f"Time={time.time() - start:.2f}s")

# Main Loop

In [74]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

print("=== Stage 1 ===")
model = BilinearCNN(num_classes=NUM_CLASSES).to(DEVICE)
import torch, numpy as np, random
seed = 87
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)


# Freeze VGG convolutional backbone
for param in model.features.parameters():
    param.requires_grad = False

optimizer = torch.optim.AdamW(
    model.fc.parameters(),
    lr=INIT_LR,
    weight_decay=WEIGHT_DECAY,
    betas=(0.9, 0.999),
    eps=1e-8
)

# StepLR: decay every 7 epochs by factor of 0.1 (same as ResNet)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

best_acc_stage1 = 0.0
for epoch in range(EPOCHS_STAGE1):
    start_time = time.time()
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion, DEVICE)
    te_loss, te_acc = evaluate(model, test_loader, criterion, DEVICE)
    scheduler.step()

    print(f"[S1-Epoch {epoch+1:02d}] Train Loss={tr_loss:.4f} | Train Acc={tr_acc*100:.2f}% | "
          f"Test Loss={te_loss:.4f} | Test Acc={te_acc*100:.2f}% | Time={time.time()-start_time:.2f}s"
          f"LR={optimizer.param_groups[0]['lr']:.2e} ")

    if te_acc > best_acc_stage1:
        best_acc_stage1 = te_acc
        torch.save(model.state_dict(), "best_bcnn_stage1_adamw.pt")

# Stage2
print("=== Stage 2 ===")

# Unfreeze all layers 
for param in model.features.parameters():
    param.requires_grad = True

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=NEW_LR,
    weight_decay=WEIGHT_DECAY,
    betas=(0.9, 0.999),
    eps=1e-8
)


scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=EPOCHS_STAGE2, gamma=0.1)

best_acc_stage2 = 0.0
for epoch in range(EPOCHS_STAGE2):
    start_time = time.time()
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion, DEVICE)
    te_loss, te_acc = evaluate(model, test_loader, criterion, DEVICE)
    scheduler.step()

    print(f"[S2-Epoch {epoch+1:02d}] Train Loss={tr_loss:.4f} | Train Acc={tr_acc*100:.2f}% | "
          f"Test Loss={te_loss:.4f} | Test Acc={te_acc*100:.2f}% | "
          f"LR={optimizer.param_groups[0]['lr']:.2e} | Time={time.time()-start_time:.2f}s")

    if te_acc > best_acc_stage2:
        best_acc_stage2 = te_acc
        torch.save(model.state_dict(), "best_bcnn_finetuned_adamw.pt")

print("Training Complete!")
print(f"Best Acc = {best_acc_stage2*100:.2f}%")

=== Stage 1 ===


[S1-Epoch 01] Train Loss=0.1635 | Train Acc=5.81% | Test Loss=0.1583 | Test Acc=34.95% | Time=9.20sLR=1.00e-04 


[S1-Epoch 02] Train Loss=0.1558 | Train Acc=28.03% | Test Loss=0.1517 | Test Acc=46.82% | Time=8.83sLR=1.00e-04 


[S1-Epoch 03] Train Loss=0.1494 | Train Acc=41.24% | Test Loss=0.1455 | Test Acc=52.14% | Time=8.86sLR=1.00e-04 


[S1-Epoch 04] Train Loss=0.1437 | Train Acc=49.68% | Test Loss=0.1397 | Test Acc=55.76% | Time=9.28sLR=1.00e-04 


...

[S2-Epoch 146] Train Loss=0.0729 | Train Acc=78.83% | Test Loss=0.0751 | Test Acc=72.21% | LR=1.00e-06 | Time=17.31s


[S2-Epoch 147] Train Loss=0.0728 | Train Acc=78.60% | Test Loss=0.0750 | Test Acc=72.25% | LR=1.00e-06 | Time=17.46s


[S2-Epoch 148] Train Loss=0.0726 | Train Acc=78.73% | Test Loss=0.0747 | Test Acc=72.40% | LR=1.00e-06 | Time=17.56s


[S2-Epoch 149] Train Loss=0.0724 | Train Acc=78.40% | Test Loss=0.0746 | Test Acc=72.39% | LR=1.00e-06 | Time=17.20s


[S2-Epoch 150] Train Loss=0.0722 | Train Acc=78.43% | Test Loss=0.0744 | Test Acc=72.23% | LR=1.00e-07 | Time=17.54s
Training Complete!
Best Acc = 72.40%
